# 1.1 - Data Assessing and Data Cleaning

___

## Project Workflow


1.0) Geological Setting and Mineral Disponibility

**1.1) Data Assessing and Data Cleaning**

1.2) Exploratory Data Analysis & Geostats

2.1) Spatial Analysis

2.2) Hydrograph Basins Delimitations

2.3) Correlation between basins and samples

3.0) Conclusion
___

## Table Of Contents

[a) Importing Libraries](#il)

[b) Importing File](#if)

[c) Data Assessing](#da)

[d) Data Cleaning](#dc)

<a name="il"></a>
## Importing Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from shapely import geometry
import matplotlib.pyplot as plt

%matplotlib inline

<a name="if"></a>
## Importing File

In [3]:
df = pd.read_csv('/content/NURE_15.csv', sep =',')
NURE_15 = df.copy()
NURE_15.head()

,rec_no,prime_id,samptyp,latitude,longitude,ornlid,site,state,quad,mapcode,...,zn_ppm,zr_ppm,po4_ppm,so4_ppm,methods,tapefile,reformat,coordprb,comments,comment2
0,5239129,8986,15,29.703,-102.759,8986,1105,TX,EMORY PEAK,NH1309,...,333,56,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,"1-38 MARAVILLAS CANYON, QUAD., 15 MIN. SAMPLE ...",
1,5239061,8874,15,29.465,-102.834,8874,1001,TX,EMORY PEAK,NH1309,...,59,64,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,SAME COORDINATES AS ORNLID(029338):,1-78 GARY HOMERSTAD % AREA MANAGER BLACK GAP W...,
2,5239062,8875,15,29.473,-102.816,8875,1002,TX,EMORY PEAK,NH1309,...,64,92,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,1-78 GARY HOMERSTAD % AREA MANAGER BLACK GAP W...,
3,5239065,8880,15,29.432,-102.863,8880,1007,TX,EMORY PEAK,NH1309,...,54,76,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,"1-38 STILLWELL CROSSING QUAD., 7.5 MIN., SAMPL...",
4,5239066,8884,15,29.871,-102.930,8884,1010,TX,EMORY PEAK,NH1309,...,19,27,0,0,"OR1, OR2, OR6, OR8",XG0386.03,HSSR SAMPLE USED IN TERRELL STUDY AREA: SAME R...,,"1-38 DOVE MTN. QUAD., 15 MIN., SAMPLE TAKEN OV...",


<a name="da"></a>
## Data Assessing

In [4]:
NURE_15.sample(1)

,rec_no,prime_id,samptyp,latitude,longitude,ornlid,site,state,quad,mapcode,...,zn_ppm,zr_ppm,po4_ppm,so4_ppm,methods,tapefile,reformat,coordprb,comments,comment2
379,5239990,29430,15,29.729,-102.79,29430,H396,TX,EMORY PEAK,NH1309,...,55,16,0,0,"OR1, OR2, OR7, OR8",XG0386.24,STILLWELL MOUNTAINS STUDY AREA:,,1-38 MARAVILLAS CANYON NE 7.5 MIN QUAD COLLECT...,


In [5]:
NURE_15.shape

(680, 137)

In [6]:
NURE_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Columns: 137 entries, rec_no to comment2
dtypes: float64(17), int64(65), object(55)
memory usage: 727.9+ KB


In [7]:
#looking for duplicated
NURE_15.duplicated().sum()

0

<a name="dc"></a>
## Data Cleaning

`1.` Tidiness Issues

- Change columns dtypes

    - Make a pattern for numbers below a certain value (e.g. <10 ppm)
    
- Filter the dataframe to only elements with high mobility

- Rename `Long__X_` and `Lat__Y_` columns


In [8]:
NURE_15.rename(columns = {'longitude':'X', 'latitude':'Y'},inplace = True)
NURE_15.head(1)

,rec_no,prime_id,samptyp,Y,X,ornlid,site,state,quad,mapcode,...,zn_ppm,zr_ppm,po4_ppm,so4_ppm,methods,tapefile,reformat,coordprb,comments,comment2
0,5239129,8986,15,29.703,-102.759,8986,1105,TX,EMORY PEAK,NH1309,...,333,56,0,0,"OR1, OR2, OR6, OR8",XG0386.03,,,"1-38 MARAVILLAS CANYON, QUAD., 15 MIN. SAMPLE ...",


In [9]:
elements = [col for col in NURE_15.columns if col.endswith(('ppm', 'pct'))]

In [10]:
NURE_15 = NURE_15[['rec_no', 'X', 'Y'] + elements]

# Displaying the filtered DataFrame
NURE_15.head()

,rec_no,X,Y,orgn_pct,u_xx_ppm,u_dn_ppm,u_fl_ppm,u_ms_ppm,u_na_ppm,ag_ppm,...,th_ppm,ti_ppm,v_ppm,w_ppm,y_ppm,yb_ppm,zn_ppm,zr_ppm,po4_ppm,so4_ppm
0,5239129,-102.759,29.703,2,0,2.25,1.44,0,0,2,...,6,2082,38,0,10,0,333,56,0,0
1,5239061,-102.834,29.465,5,0,3.05,2.32,0,0,2,...,7,2707,87,0,12,0,59,64,0,0
2,5239062,-102.816,29.473,1,0,3.18,2.03,0,0,2,...,8,3518,80,0,17,0,64,92,0,0
3,5239065,-102.863,29.432,1,0,2.84,1.78,0,0,2,...,8,2323,79,0,14,0,54,76,0,0
4,5239066,-102.930,29.871,1,0,2.14,1.46,0,0,2,...,3,1645,34,0,6,0,19,27,0,0


___

`2.` Quality Issues

- Check for the elements that are statistically significant (>= 50% in register)

- Remove `<` or `>` sign from numbers.

In [11]:
NURE_15 = NURE_15.replace(0, np.nan)

In [12]:
NURE_15.head()

,rec_no,X,Y,orgn_pct,u_xx_ppm,u_dn_ppm,u_fl_ppm,u_ms_ppm,u_na_ppm,ag_ppm,...,th_ppm,ti_ppm,v_ppm,w_ppm,y_ppm,yb_ppm,zn_ppm,zr_ppm,po4_ppm,so4_ppm
0,5239129,-102.759,29.703,2.0,NaN,2.25,1.44,NaN,NaN,2.0,...,6.0,2082.0,38.0,NaN,10.0,NaN,333.0,56.0,NaN,NaN
1,5239061,-102.834,29.465,5.0,NaN,3.05,2.32,NaN,NaN,2.0,...,7.0,2707.0,87.0,NaN,12.0,NaN,59.0,64.0,NaN,NaN
2,5239062,-102.816,29.473,1.0,NaN,3.18,2.03,NaN,NaN,2.0,...,8.0,3518.0,80.0,NaN,17.0,NaN,64.0,92.0,NaN,NaN
3,5239065,-102.863,29.432,1.0,NaN,2.84,1.78,NaN,NaN,2.0,...,8.0,2323.0,79.0,NaN,14.0,NaN,54.0,76.0,NaN,NaN
4,5239066,-102.930,29.871,1.0,NaN,2.14,1.46,NaN,NaN,2.0,...,3.0,1645.0,34.0,NaN,6.0,NaN,19.0,27.0,NaN,NaN


In [13]:
#Replacing - sample values with the absolute value of 1/2 to indicate analysis was preformed and sampled to 1/2 the detection limit


In [14]:
for col in NURE_15.select_dtypes(include=np.number).columns:  # Select numeric columns
    if col in elements:  # Check if column is in 'elements' list
        NURE_15[col] = NURE_15[col].apply(lambda x: abs(x / 2) if x < 0 else x)

In [15]:
#creating an empty list
remove = []

#iterating all elements
for e in elements:
    try:
        #counting the number of occurrences that has < or >
        not_sampled = NURE_15[e].isnull().sum()  # Use isnull() to detect NaN values

        #checking the percentage
        validity = 1 - (not_sampled/NURE_15.shape[0])

        #if the value is less than 0.5, we must remove it
        if validity < 0.5:
            del_element = e
            print(f'Remove {e}, because its value in under the proper rate {validity:0.2f}')
            #creating a list with the elements we must remove.
            remove.append(del_element)
    except:
        pass

Remove u_xx_ppm, because its value in under the proper rate 0.00
Remove u_ms_ppm, because its value in under the proper rate 0.00
Remove u_na_ppm, because its value in under the proper rate 0.00
Remove au_ppm, because its value in under the proper rate 0.00
Remove bi_ppm, because its value in under the proper rate 0.00
Remove br_ppm, because its value in under the proper rate 0.00
Remove cd_ppm, because its value in under the proper rate 0.00
Remove ce_ppm, because its value in under the proper rate 0.40
Remove cl_ppm, because its value in under the proper rate 0.00
Remove cs_ppm, because its value in under the proper rate 0.00
Remove dy_ppm, because its value in under the proper rate 0.00
Remove eu_ppm, because its value in under the proper rate 0.00
Remove f_ppm, because its value in under the proper rate 0.00
Remove hf_ppm, because its value in under the proper rate 0.33
Remove hg_ppm, because its value in under the proper rate 0.00
Remove la_ppm, because its value in under the prop

In [16]:
#filtering the dataframe
NURE_15 = NURE_15.drop(columns = remove)

In [17]:
NURE_15.shape

(680, 35)

In [20]:
from google.colab import files

In [21]:
clean = NURE_15.copy()
clean.to_csv('/content/NURE_15_data_cleaned.csv', sep = ',', index = False)
files.download('NURE_15_data_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
removed_elements_df = pd.DataFrame(remove, columns=['Removed Elements'])

# Display the table
display(removed_elements_df)

,Removed Elements
0,u_xx_ppm
1,u_ms_ppm
2,u_na_ppm
3,au_ppm
4,bi_ppm
5,br_ppm
6,cd_ppm
7,ce_ppm
8,cl_ppm
9,cs_ppm
